# Documentation Neuronal Network



In [44]:

# We will use this cell to import all the packages you will need in the following - think of it as turning on all your systems
# in your cockpit

# This makes sure that if you change code in your external scripts, they will be updated

from IPython.display import display, clear_output
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow import keras
from keras.api._v2.keras import layers
import tensorflow as tf
import os
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import datetime
import umap

cwd = os.getcwd ()

# now go ahead and Run the cell. This might take a while...
# while the cell is running, you will see ln[*] next to it. Once it finished, you will see the number of execution
# In case you want to interrupt the run of a cell, press Ctrl + C (on your german keyboard, this is Strg + C) 



### Defining the model and data parameters

Now we define our parameters to use them later


In [45]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
keep_index = [39306, 45926, 35779, 9059, 3639, 29049, 15459, 26151, 56284, 1733]

## Exploring the MNIST dataset

Before any data science or machine learning project, it is essential to get to know your data. This will enable you to detect issues, noise, pitfalls and understand what your model will learn at the end of the day. With the MNIST dataset, you will be working with a beautiful, cleaned, easy to understand, low-memory, large-scale dataset. 
Full disclaimer, MNIST is excellent for learning and research purposes, yet this is not what you can expect in real-life. 

This initially becomes obvious as MNIST is so commonly used that the Keras packages got our back with loading the data in one line. We are loading pairs of samples and ground truth annotations for both the training set (for training our model) and the test set (for testing our model). 



In [ ]:
# Loading the MNIST dataset in one line
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Printing the shape
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

## Defining our function to calculate the mse of two pictures

With this function we want to calculate the mse of two labels to get the most similar ones


In [47]:
def calculate_mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err

In [ ]:
'''keep_numbers=[]
mse_numbers = []
def searchBestImage():
  for number in range (0,10):
    mse_number = 0.0
    filter_for_number = np.where(y_train == number)[0]
    for number1 in range (len(filter_for_number)):
      for number2 in range (len(filter_for_number)):
        mse_number += calculate_mse(x_train[filter_for_number[number1]],x_train[filter_for_number[number2]])
      mse_number = mse_number / len(filter_for_number)
      mse_numbers.append(mse_number)
    min = np.argmin(mse_numbers)
    keep_index.append(filter_for_number[min])
    mse_numbers.clear()
    filter_for_number = np.where(y_train == number )[0]
'''

In [ ]:
# %%
x_train = x_train/255
x_test = x_test/255

# in the next step, we also need to reshape our input to fit our input layer later on. 
# This is due to keras expecting a definition for how many channels your input sample has, as we 
# deal with gray scale this is 1.

x_train= x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

In [ ]:
n_components=8
umap_model = umap.UMAP(n_components=n_components,)
x_train_reduced = umap_model.fit_transform(x_train)
x_test_reduced = umap_model.transform(x_test)
#print("Number of input:",lda.n_components_)
keep_index = [39306, 45926, 35779, 9059, 3639, 15459, 26151, 56284]

In [ ]:
x_train_reduced=x_train_reduced[keep_index]
y_train=y_train[keep_index]
print("y_train:",y_train)
print('x_red:', x_train_reduced.shape)

In [39]:
marvin = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(n_components,)),
  tf.keras.layers.Dense(256, activation='sigmoid',kernel_initializer='glorot_uniform'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(128, activation='sigmoid',kernel_initializer='glorot_uniform'),
  tf.keras.layers.Dropout(0.1),#
  tf.keras.layers.Dense(10, activation='softmax',kernel_initializer='glorot_uniform', use_bias=True)
])
marvin.summary()

In [ ]:
marvin.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss= 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#%%
early_stopping = tf.keras.callbacks.EarlyStopping (monitor='val_accuracy',patience=15,mode="max") 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau (monitor ='val_accuracy', factor =0.5 , min_delta=0.00005 , 
                                                  patience=6, min_lr=0.0001, mode="max")
model_checkpoint = tf.keras.callbacks.ModelCheckpoint (filepath=os.path.join(cwd,'models/DNN_UMAP_30.h5'),
                                                       monitor='val_accuracy',save_best_only=True, mode="max")
#%%

In [ ]:
marvin.fit(
    x_train_reduced,
    y_train,
    epochs= 200,
    batch_size= 32,
    validation_data=(x_test_reduced, y_test),
    callbacks=[model_checkpoint,reduce_lr]
)

In [ ]:
#SAVE MODEL 
model_name = 'model_1'
marvin.save(model_name, save_format='h5')

print('Success! You saved Marvin as: ', model_name)

In [ ]:
#%% EVALUATE MODEL (just to be sure the last val acc value was right)
marvin_reloaded = tf.keras.models.load_model('models/DNN_UMAP_30.h5')
marvin_reloaded.summary()
loss_and_metrics = marvin_reloaded.evaluate(x_test_reduced, y_test, verbose=2)

print("Test Loss", loss_and_metrics[0])
print("Test Accuracy", loss_and_metrics[1])

#%%

# Let Marvin predict on the test set, so we have some data to evaluate his performance.
predictions = marvin_reloaded.predict([x_test_reduced])

# Remember that the prediction of Marvin is a probability distribution over all ten-digit classes
# We want him to assign the digit class with the highest probability to the sample.
predictions = np.argmax(predictions, axis=1)
#pd.DataFrame(predictions)